In [ ]:
from nuscenes import NuScenes
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.models as tvmodels
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import *
# from torch.distributions.multivariate_normal import MultivariateNormal

In [ ]:
# DATAROOT = '../nu_data/v1.0-mini/'
# nuscenes = NuScenes('v1.0-mini', dataroot=DATAROOT)
# from nuscenes.eval.prediction.splits import get_prediction_challenge_split
# mini_train = get_prediction_challenge_split("mini_train", dataroot=DATAROOT)

In [ ]:
# from nuscenes.prediction import PredictHelper
# helper = PredictHelper(nuscenes)

In [ ]:
# instance_token, sample_token = mini_train[0].split("_")
# annotation = helper.get_annotations_for_sample('7626dde27d604ac28a0240bdd54eba7a')

In [ ]:
# future_xy_local = helper.get_future_for_agent(instance_token, sample_token, seconds=3, in_agent_frame=True)
# future_xy_global = helper.get_future_for_agent(instance_token, sample_token, seconds=3, in_agent_frame=False)
# helper.get_future_for_agent(instance_token, sample_token, seconds=3, in_agent_frame=True, just_xy=False)
# sample = helper.get_annotations_for_sample(sample_token)

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# from nuscenes.prediction.input_representation.static_layers import StaticLayerRasterizer
# from nuscenes.prediction.input_representation.agents import AgentBoxesWithFadedHistory
# from nuscenes.prediction.input_representation.interface import InputRepresentation
# from nuscenes.prediction.input_representation.combinators import Rasterizer

# static_layer_rasterizer = StaticLayerRasterizer(helper)
# agent_rasterizer = AgentBoxesWithFadedHistory(helper, seconds_of_history=2)
# mtp_input_representation = InputRepresentation(static_layer_rasterizer, agent_rasterizer, Rasterizer())

# instance_token_img, sample_token_img = 'bc38961ca0ac4b14ab90e547ba79fbb6', '7626dde27d604ac28a0240bdd54eba7a'
# anns = [ann for ann in nuscenes.sample_annotation if ann['instance_token'] == instance_token_img]
# img = mtp_input_representation.make_input_representation(instance_token_img, sample_token_img)

In [ ]:
# import glob
# import os
# def sortKeyFunc(s):
#     return int(os.path.basename(s)[0:-4])
# video_dictionary = glob.glob(path + 'map_valid/*.png')
# video_dictionary.sort(key=sortKeyFunc)
# np.save('map_valid.npy', video_dictionary)

# 1. Data

In [ ]:
path = 'D:/Nuscenes/MHA_JAM/'
input_train = np.load(path + 'map_train.npy', allow_pickle=True)
input_valid = np.load(path + 'map_valid.npy', allow_pickle=True)
trajectory_train = np.load(path + 'trajectories_train.npy', allow_pickle=True)
trajectory_valid = np.load(path + 'trajectories_valid.npy', allow_pickle=True)

In [ ]:
num_train = len(trajectory_train)
num_valid = len(trajectory_valid)

# 2. Model 

In [ ]:
class args():
    def __init__(self):
        self.top_len = 40
        self.bot_len = 10
        self.left_len = 25
        self.right_len = 25
        self.grid_size = 28
        self.map_size = 50
        self.input_size = 5
        self.encoder_hidden_size = 64
        self.embedding_size = 32
        self.image_size = 500
        self.image_out_size = 28
        self.image_out_chanel = 512
        self.head_n = 16
        self.key_dim = 64
        self.decoder_hidden_size = 128
        self.prediction_time = 6 # 12s for prediction
        self.decoder_output_size = 2 # 4 parameter define guassian distribution

In [ ]:
# def format_input(relevent_traj_list, len_expected_history):
#     '''
#     1. padding the not enough history with oldest value
#     2. padding none with 0
#     '''
#     input_li = list(np.array(list(relevent_traj_list[0].values())[0][0]))
#     # print('input_li', input_li)
#     for key, value in relevent_traj_list[1].items():
        
#         new_item = np.array(items[1][0])
#         len_item = len(items[1][0])
#         if len_item < len_expected_history:
#             holder_item = np.zeros((len_expected_history, new_item.shape[1]))
#             holder_item[len_expected_history - len_item:,:] = new_item
#             for i in range(len_expected_history - len_item):
#                 holder_item[i, :] = new_item[0]
#             new_item = holder_item
#         print(input_li)
#         input_li.append(new_item[-len_expected_history:])
#         print(np.array(new_item[-len_expected_history:]))
#     return np.nan_to_num(np.array(input_li)).transpose((1,0,2))

In [ ]:
def get_current_coordinate(relevent_traj_list):
    '''
    format - self.O_corr [('bc38961ca0ac4b14ab90e547ba79fbb6', [392.945, 1148.426])]
    format - self.T_corr [('bc38961ca0ac4b14ab90e547ba79fbb6', [392.945, 1148.426]), 
    # ('bc38961ca0ac4b14ab90e547ba79fbb6', [392.945, 1148.426])]
    '''
    if list(relevent_traj_list[0].values()) != [None]:
        target_corr = [(i[0], i[1][0][-1][:2]) for i in relevent_traj_list[0].items()]
        other_corr = [(i[0], i[1][0][-1][:2]) for i in relevent_traj_list[1].items()]
        return target_corr, other_corr
    else:
        return None, None

In [ ]:
def format_input(relevent_traj_list):
    past = []
    future = []
    for key, value in relevent_traj_list[0].items():
        past.append(torch.from_numpy(np.nan_to_num(np.array(value[0])).astype(np.float32)))
        future.append(torch.from_numpy(np.nan_to_num(np.array(value[1])).astype(np.float32)))
    for key, value in relevent_traj_list[1].items():
        past.append(torch.from_numpy(np.nan_to_num(np.array(value[0])).astype(np.float32)))
        future.append(torch.from_numpy(np.nan_to_num(np.array(value[1])).astype(np.float32)))
    return past, future

In [ ]:
def compute_grid(args, target_corr, other_corr):
    '''
    return mask: shape[other_n, grid_size ** 2]
    '''
    top_len = args.top_len
    bot_len = args.bot_len
    left_len = args.left_len
    right_len = args.right_len
    grid_size = args.grid_size

    target_x, target_y = target_corr[0][1]
    height_low = target_y - bot_len
    width_low = target_x - left_len
    mask = np.zeros((len(other_corr), grid_size**2))
    
    idx = 0
    for item in other_corr:
        other_x, other_y = item[1]
        cell_x = int(np.floor(((other_x - width_low)/ args.map_size) * grid_size))
        cell_y = int(np.floor(((other_y - height_low)/ args.map_size) * grid_size))
        mask[idx, cell_x + cell_y * grid_size] = 1
        idx += 1
    return mask

In [ ]:
class SocialModel(nn.Module):
    def __init__(self, args, infer=False):
        super(SocialModel, self).__init__()
        self.input_size = args.input_size
        self.hidden_size = args.encoder_hidden_size
        self.embedding_size = args.embedding_size
        self.grid_size = args.grid_size
        
        self.input_embedding_layer = nn.Linear(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, batch_first=True)
        
    def get_social_tensor(self, grid, other_hidden_states):
        '''
        grid: shape[other_vehicle_n, grid_size ** 2]
        hidden_states: shape[other_vehicle_n, hidden_size]
        
        return: shape[grid_size ** 2， hidden_size]
        '''
        other_vehicle_n = grid.size()[0]
        social_tensor = torch.mm(torch.t(grid), other_hidden_states)
        social_tensor = social_tensor.view(self.grid_size, self.grid_size, self.hidden_size)
        return social_tensor.permute(2, 0, 1).unsqueeze(0)
    
    def forward(self, x, grid, lens):
        '''
        x: shape[history_len (6s 12items), batch_size(other_n), state_n (5)]
        grid: shape[other_vehicle_n, grid_size ** 2]
        '''
        x = self.input_embedding_layer(x)
        packed_x = pack_padded_sequence(x, lens, batch_first=True, enforce_sorted=False)
        packed_out, (hidden, cell) = self.lstm(packed_x)
        target_hidden, other_hidden = hidden[0,0], hidden[0,1:]
        return target_hidden, self.get_social_tensor(grid, other_hidden)# , embedding

In [ ]:
def Mapmodel(args):
    image_size = args.image_size
    out_size = args.image_out_chanel
    resnet50 = tvmodels.resnet50(pretrained=True)
    resnet50 = nn.Sequential(*list(tvmodels.resnet50(pretrained=True).children())[:-3])
    resnet50.output = nn.Conv2d(1024, 512, kernel_size=3)
    resnet50.output_relu = nn.ReLU()
    resnet50.output2 = nn.Conv2d(512, 512, kernel_size=3)
    return resnet50

In [ ]:
class DecoderModel(nn.Module):
    def __init__(self, args):
        super(DecoderModel, self).__init__()
        self.prediction_time = args.prediction_time
        self.decoder_hidden_size = args.decoder_hidden_size
        self.encoder_hidden_size = args.encoder_hidden_size
        self.decoder_output_size = args.decoder_output_size
        # self.embedding_size = args.embedding_size
        self.input_size = args.input_size
        
        # self.embedding_layer = input_embedding_layer
        self.lstm1 = nn.LSTMCell(input_size=self.encoder_hidden_size
                                 , hidden_size=self.decoder_hidden_size)
        self.fc = nn.Linear(self.decoder_hidden_size, self.decoder_output_size)
        
    def forward(self, initial, attention_feature):
        '''
        most_recent_state [targetx, targety, 0,0,0,0]
        '''
        # initial_embed = self.embedding_layer(initial)
        # initial_repeat = initial_embed.repeat(attention_feature.shape[0]).view(attention_feature.shape[0], -1)
        # lstm_input = torch.cat((initial_repeat, attention_feature), 1)
        hx = torch.randn(attention_feature.shape[0], self.decoder_hidden_size).to(device)
        cx = torch.randn(attention_feature.shape[0], self.decoder_hidden_size).to(device)
        output = []
        
        for i in range(self.prediction_time):
            hx, cx = self.lstm1(attention_feature, (hx, cx))
            temp_output = self.fc(hx)
            output.append(temp_output)
            # embed_output = self.embedding_layer(temp_output)
            # lstm_input = torch.cat((embed_output, attention_feature), 1)
            
        return torch.stack(output, dim=1)

In [ ]:
class VisualAttention(nn.Module):
    def __init__(self):
        super(VisualAttention, self).__init__()
    def forward(self, query, key, value, args):
        key = key.view(args.encoder_hidden_size, -1).squeeze()
        QK = torch.mm(query, key).view(-1)
        alpha = F.softmax(QK / np.sqrt(args.key_dim)).view(args.image_out_size, args.image_out_size)
        a = alpha * value
        return torch.sum(a, dim=(2,3))

In [ ]:
class MHA_JAM(nn.Module):
    def __init__(self, args):
        super(MHA_JAM, self).__init__()
        
        self.args = args
        self.image_out_size = args.image_out_size
        self.image_out_channel = args.image_out_chanel
        self.head_n = args.head_n
        self.key_dim = args.key_dim
        self.encoder_hidden_size = args.encoder_hidden_size
        
        self.social_model = SocialModel(args)
        self.map_model = Mapmodel(args)
        self.decoder_model = DecoderModel(args) # Not Implemented
        
        # self.gen_query = [nn.Linear(self.encoder_hidden_size, self.encoder_hidden_size * ) for i in range(self.head_n)]
        # self.gen_key = [nn.Conv2d(self.image_out_channel + self.encoder_hidden_size, self.key_dim, kernel_size=1) 
        #                 for i in range(self.head_n)]
        # self.gen_value = [nn.Conv2d(self.image_out_channel + self.encoder_hidden_size, self.key_dim, kernel_size=1) 
        #                  for i in range(self.head_n)]
        # self.attentions = [VisualAttention() for i in range(self.head_n)]
        
        self.gen_query = nn.Linear(self.encoder_hidden_size, self.encoder_hidden_size * self.head_n)
        self.gen_key = nn.Conv2d(self.image_out_channel + self.encoder_hidden_size, 
                                 self.key_dim * self.head_n, kernel_size=1)
        self.gen_value = nn.Conv2d(self.image_out_channel + self.encoder_hidden_size, 
                                   self.key_dim * self.head_n, kernel_size=1)
        self.attentions = [VisualAttention() for i in range(self.head_n)]
        
        self.p_prediction_fc1 = nn.Linear(self.encoder_hidden_size*self.head_n, self.head_n)
        # self.p_prediction_fc2 = nn.Linear(int(self.encoder_hidden_size / 2), 1)
        
    def forward(self, map_image, grid, trajectory_data, lens):
        '''
        map_image is shape(1, 3, 500, 500)
        grid tell the occupancy info around the vehicle
        trajectory_data tell the trajectory we use 
        '''
        map_feature = self.map_model(map_image)
        target_hidden, social_feature = self.social_model(trajectory_data, grid, lens)
        social_feature = torch.flip(social_feature, [2])
        #??? the map and socal may have different direction 
        concat_feature = torch.cat((map_feature, social_feature), 1)
        query = self.gen_query(target_hidden).unsqueeze(0)
        key = self.gen_key(concat_feature)
        value = self.gen_value(concat_feature)
        attention_feature_list = []
        for i in range(self.head_n): # [1, 1024, 28, 28]
            query_i = query[:, i * self.encoder_hidden_size: (i+1) * self.encoder_hidden_size]
            key_i = key[:, i * self.key_dim: (i+1) * self.key_dim ,: ,:]
            value_i = value[:, i * self.key_dim: (i+1) * self.key_dim ,: ,:]
            attention_feature_list.append(target_hidden + self.attentions[i](query_i, key_i, value_i, self.args))
        
        initial = trajectory_data[0][0]
        cated_attention_feature = torch.cat(attention_feature_list)
#         cated_attention_feature_for_pred = cated_attention_feature.clone()
        output = self.decoder_model(initial, cated_attention_feature)
        # for probability prediction for each interaction
        p_pred = F.softmax(self.p_prediction_fc1(cated_attention_feature.view(-1)), dim=0)
        # p_pred = self.p_prediction_fc2(p_pred)
        return output, p_pred

In [ ]:
# # initialize all the hyper-parameter
# a = args()
# # get all the agent's coordination in this timestep
# target_corr, other_corr = get_current_coordinate(relevent_traj_list = 
#                        agent_rasterizer.extract_relevant_trajectory(instance_token_img, sample_token_img))
# # for a given time and agent, compute the occupancy grid
# grid = compute_grid(a, target_corr, other_corr)
# # compute the input history data
# relevent_traj_list = agent_rasterizer.extract_relevant_trajectory(instance_token_img, sample_token_img)
# input_data = format_input(relevent_traj_list, 5)
# # compute input image
# img = mtp_input_representation.make_input_representation(instance_token_img, sample_token_img)
# # model forward
# model = MHA_JAM(a)
# output, p_pred = model(torch.tensor(img).permute(2, 0, 1).unsqueeze(0).float(), 
#       torch.tensor(grid).float(), 
#       torch.tensor(input_data).float())
# p_pred

# 3. Loss function

In [ ]:
# def process_multivariate_Gaussian(output, k=2, alpha=0.05, eps=1e-3):
#     """
#     Format neural network output into mean and covariance
#     Args:
#     output : (k*(k + 3)/2) tensor
#     k : integer dimension of regression problem
#     alpha : saturation coefficient
#     eps : small value for numerical stability
#     """
#     assert output.shape[0] == k*(k + 3)//2
#     mean = output[:k]
#     var = output[k:2*k].exp()
#     var_mat = torch.sqrt(var.unsqueeze(1)*var.unsqueeze(0))
#     # Build correlation matrix
#     rhos = (1 - eps)*torch.tanh(alpha*output[2*k:])
#     rho_mat = torch.ones_like(var_mat)
#     for i in range(k):
#         for j in range(i + 1, k):
#             # Flattened upper diagonal indexing
#             rho = rhos[k*i - i*(i + 3)//2 + j - 1]
#             rho_mat[i, j] = rho
#             rho_mat[j, i] = rho
#     return mean, rho_mat*var_mat

In [ ]:
# def loss_backward(input_data, target, p):
#     '''
#     input data (head_n, timestep, 6): each timestep 6 dimension (mu_x, mu_y, v_x, v_y)
#     target data (timestep, 2): each timestep 2 dimension (x, y)
#     p (head_n): 
#     '''
#     sum_prob = []
#     min_idx = 0
#     for head_idx in range(input_data.shape[0]):
        
# #         dist = [MultivariateNormal(input_data[head_idx, i, :2], 
# #                                    torch.diag(input_data[head_idx, i, 2:4].exp())) for i in range(input_data.shape[1])]
#         dist = []
#         for i in range(input_data.shape[1]):
#             mean, cov = process_multivariate_Gaussian(input_data[head_idx, i])
#             dist.append(MultivariateNormal(mean, cov))
        
#         prob = torch.stack([- dist[i].log_prob(target[i, :]) 
#                             for i in range(input_data.shape[1])])
#         sum_prob.append(prob.sum())
        
#     min_val = torch.min(torch.stack(sum_prob))
#     min_idx = torch.argmin(torch.stack(sum_prob))
#     prob = - torch.log(p[min_idx] + 1e-7)
#     loss = min_val + prob
#     return loss, input_data[min_idx,:,:2].detach().cpu().numpy()

In [ ]:
def loss_backward(input_data, target, p):
    '''
    input data (head_n, timestep, 4): each timestep 4 dimension (mu_x, mu_y, v_x, v_y)
    target data (timestep, 2): each timestep 2 dimension (x, y)
    p (head_n): 
    '''
#     mean, logvar = input_data[:,:,0:2], input_data[:,:,2:4]
#     mse_loss = (torch.square(mean - target)*torch.exp(-logvar)).sum(dim=-1).sum(dim=-1)
#     var_loss = logvar.sum(dim=-1).sum(dim=-1)
#     loss = mse_loss + var_loss
    
    mse = torch.square(input_data - target).sum(dim=-1).sum(dim=-1)
    loss = mse
    loss_min = torch.min(loss)
    loss_index = torch.argmin(loss)
    
    prob = - torch.log(p[loss_index] + 1e-7)
    loss = loss_min + prob
    return loss, input_data[loss_index,:,:2].detach().cpu().numpy()

In [ ]:
device = torch.device('cuda')
num_epoch = 100
a = args()
model = MHA_JAM(a)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
# model.load_state_dict(torch.load('MHA_JAM.pth'))

In [ ]:
loss_epoch_train = []
loss_epoch_valid = []
for epoch in range(num_epoch):
    index = np.random.permutation(num_train)
    running_loss = 0
    for n in range(num_train):
        model.train()
        i = index[n]
        target_corr, other_corr = get_current_coordinate(relevent_traj_list = trajectory_train[i])
        if target_corr == None:
            print('skip', i)
            continue
        # for a given time and agent, compute the occupancy grid
        grid = compute_grid(a, target_corr, other_corr)
        # compute the input history data
#         length = len(list(trajectory_train[i][0].values())[0])
#         trajectory = format_input(trajectory_train[i], length)
        trajectory, targets = format_input(trajectory_train[i])
        # compute input image
        lengths = torch.LongTensor([len(item) for item in trajectory])
        trajectory = pad_sequence(trajectory, batch_first=True)
        
        image = plt.imread(input_train[i])
        # model forward

        output, p_pred = model(torch.FloatTensor(image).permute(2, 0, 1).unsqueeze(0).to(device), 
                               torch.FloatTensor(grid).to(device),
                               trajectory.to(device),
                               lengths.to(device))

        optimizer.zero_grad()
        loss, best_prediction = loss_backward(output, targets[0][0:6,0:2].to(device), p_pred)
        loss.backward()
        optimizer.step()
        running_loss += loss.detach().cpu().numpy()
        
        if (n+1) % 1000 == 0:
            print(epoch+1, n+1, loss.detach().cpu().numpy(), i+1)
        
    running_loss /= num_train
    loss_epoch_train.append(running_loss)
    torch.save(model.state_dict(), 'MHA_JAM_regression.pth')
    print('Train_loss:', running_loss)
    
    print('validation')
    running_loss = 0
    for i in range(num_valid):
        model.eval()
        target_corr, other_corr = get_current_coordinate(relevent_traj_list = trajectory_valid[i])
        if target_corr == None:
            print('skip')
            continue
        # for a given time and agent, compute the occupancy grid
        grid = compute_grid(a, target_corr, other_corr)
        # compute the input history data
#         length = len(list(trajectory_train[i][0].values())[0])
#         trajectory = format_input(trajectory_train[i], length)
        trajectory, targets = format_input(trajectory_valid[i])
        # compute input image
        lengths = torch.LongTensor([len(item) for item in trajectory])
        trajectory = pad_sequence(trajectory, batch_first=True)
        
        image = plt.imread(input_valid[i])
        # model forward
        with torch.no_grad():
            output, p_pred = model(torch.FloatTensor(image).permute(2, 0, 1).unsqueeze(0).to(device), 
                               torch.FloatTensor(grid).to(device),
                               trajectory.to(device),
                               lengths.to(device))

            loss, best_prediction = loss_backward(output, targets[0][0:6,0:2].to(device), p_pred)
            running_loss += loss.detach().cpu().numpy()
            
            if (i+1) % 1000 == 0:
                print(epoch+1, i+1, loss.detach().cpu().numpy())
            
#         if (i+1) % 10 == 0:
#             plt.scatter(best_prediction[:,0], best_prediction[:,1], color='r')
#             plt.scatter(target_valid[i,:,0], target_valid[i,:,1], color='b')
#             plt.show()
    running_loss /= num_valid
    loss_epoch_valid.append(running_loss)
    print('Valid_loss:', running_loss)

In [ ]:
for i in range(num_valid):
    target_corr, other_corr = get_current_coordinate(relevent_traj_list = trajectory_valid[i])
    if target_corr == None:
        print('skip')
        continue
    # for a given time and agent, compute the occupancy grid
    grid = compute_grid(a, target_corr, other_corr)
    # compute the input history data
#         length = len(list(trajectory_train[i][0].values())[0])
#         trajectory = format_input(trajectory_train[i], length)
    trajectory, targets = format_input(trajectory_valid[i])
    # compute input image
    lengths = torch.LongTensor([len(item) for item in trajectory])
    trajectory = pad_sequence(trajectory, batch_first=True)

    image = plt.imread(input_valid[i])
    # model forward
    with torch.no_grad():
        output, p_pred = model(torch.FloatTensor(image).permute(2, 0, 1).unsqueeze(0).to(device), 
                           torch.FloatTensor(grid).to(device),
                           trajectory.to(device),
                           lengths.to(device))

        loss, best_prediction = loss_backward(output, targets[0][:,0:2].to(device), p_pred)
        running_loss += loss.detach().cpu().numpy()

        if (i+1) % 1000 == 0:
            print(epoch+1, i+1, loss.detach().cpu().numpy())
            
        if (i+1) % 10 == 0:
            plt.scatter(best_prediction[:,0], best_prediction[:,1], color='r')
            plt.scatter(targets[0][:,0], targets[0][:,1], color='b')
            plt.show()
    running_loss /= num_valid
    loss_epoch_valid.append(running_loss)
    print('Valid_loss:', running_loss)

In [ ]:
# # for i in range(num_train):
# i = 0
# position = []
# for key, value in trajectory_train[i][0].items():
#     for n in range(len(trajectory_train[i][0][key])):
#         position.append(trajectory_train[i][0][key][n][0:2])
# position = np.array(position)
# position

In [ ]:
# plt.plot(position[:,0], position[:,1], color='r')
# plt.plot(target_train[i,:,0], target_train[i,:,1], color='b')
# plt.show()

# 4. Social tensor visualization

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm
import matplotlib.colorbar
import matplotlib.colors

def cuboid_data(center, size=(1,1,1)):
    # code taken from
    # http://stackoverflow.com/questions/30715083/python-plotting-a-wireframe-3d-cuboid?noredirect=1&lq=1
    # suppose axis direction: x: to left; y: to inside; z: to upper
    # get the (left, outside, bottom) point
    o = [a - b / 2 for a, b in zip(center, size)]
    # get the length, width, and height
    l, w, h = size
    x = [[o[0], o[0] + l, o[0] + l, o[0], o[0]],  # x coordinate of points in bottom surface
         [o[0], o[0] + l, o[0] + l, o[0], o[0]],  # x coordinate of points in upper surface
         [o[0], o[0] + l, o[0] + l, o[0], o[0]],  # x coordinate of points in outside surface
         [o[0], o[0] + l, o[0] + l, o[0], o[0]]]  # x coordinate of points in inside surface
    y = [[o[1], o[1], o[1] + w, o[1] + w, o[1]],  # y coordinate of points in bottom surface
         [o[1], o[1], o[1] + w, o[1] + w, o[1]],  # y coordinate of points in upper surface
         [o[1], o[1], o[1], o[1], o[1]],          # y coordinate of points in outside surface
         [o[1] + w, o[1] + w, o[1] + w, o[1] + w, o[1] + w]]    # y coordinate of points in inside surface
    z = [[o[2], o[2], o[2], o[2], o[2]],                        # z coordinate of points in bottom surface
         [o[2] + h, o[2] + h, o[2] + h, o[2] + h, o[2] + h],    # z coordinate of points in upper surface
         [o[2], o[2], o[2] + h, o[2] + h, o[2]],                # z coordinate of points in outside surface
         [o[2], o[2], o[2] + h, o[2] + h, o[2]]]                # z coordinate of points in inside surface
    return np.array(x), np.array(y), np.array(z)

def plotCubeAt(pos=(0,0,0), c="w", alpha=0.1, ax=None):
    # Plotting N cube elements at position pos
    if ax !=None:
        X, Y, Z = cuboid_data( (pos[0],pos[1],pos[2]) )
        # print(c)
        ax.plot_surface(X, Y, Z, color=c, rstride=1, cstride=1, alpha=alpha)

def plotMatrix(ax, x, y, z, data, cmap=plt.cm.get_cmap('PiYG'), cax=None, alpha=0.1):
    # plot a Matrix 
    norm = matplotlib.colors.Normalize(vmin=-0.2, vmax=0.2)
    colors = lambda i,j,k : (1,1,1,alpha) if (0.01<=data[i,j,k]<= 0.01) else matplotlib.cm.ScalarMappable(norm=norm,cmap = cmap).to_rgba(data[i,j,k])
    #print(colors)
    for i, xi in enumerate(x):
            for j, yi in enumerate(y):
                for k, zi, in enumerate(z):
                    # print(colors(i,j,k))
                    plotCubeAt(pos=(xi, yi, zi), c=colors(i,j,k), alpha=alpha,  ax=ax)



    if cax !=None:
        cbar = matplotlib.colorbar.ColorbarBase(cax, cmap=plt.cm.get_cmap('bwr'), norm=norm, orientation='vertical')  
        cbar.set_ticks(np.unique(data))
        # set the colorbar transparent as well
        cbar.solids.set(alpha=alpha)              

# x and y and z coordinates
x = np.array(range(28))
y = np.array(range(28))
z = np.array(range(64))
data = social.detach().numpy()

data[data < 0] = -0.2
data[data > 0] = 0.2
data[data == 0] = 0

fig = plt.figure(figsize=(10,7))
ax = fig.add_axes([0.1, 0.1, 0.7, 0.8], projection='3d')
ax_cb = fig.add_axes([0.8, 0.7, 0.05, 0.45])

plotMatrix(ax, x, y, z, data[:28,:28,:64], cmap="bwr", cax = ax_cb)

plt.show()

In [ ]:
t = []
for n in range(num_valid):
    target_corr, other_corr = get_current_coordinate(relevent_traj_list = trajectory_valid[n])
    if target_corr == None:
        t.append(n)
        print('skip', n)